<a href="https://colab.research.google.com/github/shakil1819/NLTK-LSTM-Based-Hate-Speech-Detection/blob/main/LSTM_Twitter_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imorting Libraries

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import WordNetLemmatizer
import itertools
from wordcloud import WordCloud
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM, SpatialDropout1D, Embedding
from tensorflow.keras import utils
from tensorflow.keras.utils import to_categorical
from joblib import dump, load

# Reading the dataset

In [2]:
text = []
clas = []
df = pd.read_csv('https://raw.githubusercontent.com/shakil1819/NLTK-LSTM-Based-Hate-Speech-Detection/main/Dataset/labeled_data.csv')
text = df['tweet'].tolist()
clas = df['class'].tolist()
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


# creating a new dataframe for easy text processing

In [3]:
df = pd.DataFrame({'tweet': text, 'class': clas})

# Finding if there is any missing data

In [4]:
print(df.isnull().sum())

tweet    0
class    0
dtype: int64


# Converting the data into lower case.

In [5]:
df['tweet'] = df['tweet'].apply(lambda x:x.lower())

# removing punctuations

In [6]:
punctuation_signs = list("?:!.,;")
df['tweet'] = df['tweet']

for punct_sign in punctuation_signs:
    df['tweet'] = df['tweet'].str.replace(punct_sign, '')

C:\Users\USER\AppData\Local\Temp\ipykernel_11936\958571737.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['tweet'] = df['tweet'].str.replace(punct_sign, '')


# Removing '\n' and '\t', extra spaces, quoting text, and progressive pronouns.

In [7]:
df['tweet'] = df['tweet'].apply(lambda x: x.replace('\n', ' '))
df['tweet'] = df['tweet'].apply(lambda x: x.replace('\t', ' '))
df['tweet'] = df['tweet'].str.replace("    ", " ")
df['tweet'] = df['tweet'].str.replace('"', '')
df['tweet'] = df['tweet'].str.replace("'s", "")

# removing stop-words

In [8]:
nltk.download('stopwords')
stop_words = list(stopwords.words('english'))
for stop_word in stop_words:
    regex_stopword = r"\b" + stop_word + r"\b"
    df['tweet'] = df['tweet'].str.replace(regex_stopword, '')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\USER\AppData\Local\Temp\ipykernel_11936\1953869578.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df['tweet'].str.replace(regex_stopword, '')


# Using Bag of Words approach for final data Preparation

In [9]:
cv = CountVectorizer(max_features = 75)
X = cv.fit_transform(df['tweet']).toarray()
y = df['class']

# Splitting the Data using Stratified split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify=y, random_state = 42)

# Converting the labels into categorical format

In [11]:
y_train=to_categorical(y_train, num_classes = 3, dtype='float32')
y_test=to_categorical(y_test, num_classes = 3, dtype='float32')

# Creating and Training an LSTM Model

In [16]:
# model = Sequential()
# model.add(Embedding(232337, 100, input_length=X_train.shape[1]))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(20, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(3, activation='softmax'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# epochs = 25
# batch_size = 64

# history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/25
272/272 [==============================] - 46s 163ms/step - loss: 0.4293 - accuracy: 0.7676 - val_loss: 0.4022 - val_accuracy: 0.7743
Epoch 2/25
272/272 [==============================] - 44s 162ms/step - loss: 0.4021 - accuracy: 0.7743 - val_loss: 0.4015 - val_accuracy: 0.7743
Epoch 3/25
272/272 [==============================] - 44s 161ms/step - loss: 0.4015 - accuracy: 0.7743 - val_loss: 0.4008 - val_accuracy: 0.7743
Epoch 4/25
272/272 [==============================] - 44s 162ms/step - loss: 0.4010 - accuracy: 0.7743 - val_loss: 0.3997 - val_accuracy: 0.7743
Epoch 5/25
272/272 [==============================] - 44s 161ms/step - loss: 0.3971 - accuracy: 0.7743 - val_loss: 0.3911 - val_accuracy: 0.7743
Epoch 6/25
272/272 [==============================] - 44s 162ms/step - loss: 0.3926 - accuracy: 0.7743 - val_loss: 0.3901 - val_accuracy: 0.7743
Epoch 7/25
272/272 [==============================] - 44s 161ms/step - loss: 0.3901 - accuracy: 0.7745 - val_loss: 0.3710 - val_ac

# Saving the LSTM Model

In [19]:
# model.save('lstm.h5')

c:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Loading the LSTM Model

In [12]:
from keras.models import load_model

loaded_model = load_model('lstm.h5')

# Prediction

In [25]:
# Function to preprocess text for prediction
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation
    text = ''.join([char for char in text if char not in '?:!.,;'])
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text


# Example text for prediction
new_text = "I hate University"

# Preprocess the text
preprocessed_text = preprocess_text(new_text)

# Tokenize the preprocessed text using the same CountVectorizer used during training
tokenized_text = cv.transform([preprocessed_text]).toarray()

# Make predictions using the loaded LSTM model
predictions = loaded_model.predict(tokenized_text)

# Convert the predictions to class labels
# predicted_class = np.argmax(predictions, axis=1)

# Print the predicted class
print("Predicted Class:", predictions)


1/1 [==============================] - 0s 18ms/step
Predicted Class: [[0.02518461 0.8621576  0.11265778]]
